In [1]:
#Imports
import nfl_data_py as nfl
import pandas as pd
from functions import import_schedule_data, import_weekly_performance_data, load_and_process_kicker_data, load_and_process_defense_data
from functions import scrape_adp_data
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
# Weekly performance data
# Define the years for which you want to import and filter data
years = [2019, 2020, 2021]

# Call the functions to get the filtered data
weekly_performance_2019 = import_weekly_performance_data([2019])
weekly_performance_2020 = import_weekly_performance_data([2020])
weekly_performance_2021 = import_weekly_performance_data([2021])

Downcasting floats.
Downcasting floats.
Downcasting floats.


In [3]:
# Kicker data
# Get Roster data to enrich with Kicker data
roster_2019 = nfl.import_seasonal_rosters([2019])
roster_2020 = nfl.import_seasonal_rosters([2020])
roster_2021 = nfl.import_seasonal_rosters([2021])

# Load and process Kicker data for each year
full_kicker_2019 = load_and_process_kicker_data(2019, roster_2019)
full_kicker_2020 = load_and_process_kicker_data(2020, roster_2020)
full_kicker_2021 = load_and_process_kicker_data(2021, roster_2021)

c:\Users\Lenny\seminar_paper_flath\datadriven-fantasyfootball\fantasy_football_data\functions.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  kicker = pd.read_html(str(table))[0]
c:\Users\Lenny\seminar_paper_flath\datadriven-fantasyfootball\fantasy_football_data\functions.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  kicker = pd.read_html(str(table))[0]
c:\Users\Lenny\seminar_paper_flath\datadriven-fantasyfootball\fantasy_football_data\functions.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  kicker = pd.read_html(str(table))[0]


In [4]:
# Defense data
# Load and process Defense data for each year
full_defense_2019 = load_and_process_defense_data(2019)
full_defense_2020 = load_and_process_defense_data(2020)
full_defense_2021 = load_and_process_defense_data(2021)

c:\Users\Lenny\seminar_paper_flath\datadriven-fantasyfootball\fantasy_football_data\functions.py:63: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  defense = pd.read_html(str(table))[0]
c:\Users\Lenny\seminar_paper_flath\datadriven-fantasyfootball\fantasy_football_data\functions.py:63: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  defense = pd.read_html(str(table))[0]
c:\Users\Lenny\seminar_paper_flath\datadriven-fantasyfootball\fantasy_football_data\functions.py:63: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  defense = pd.read_html(str(table))[0]


In [5]:
# Define the years for which you have data
years = [2019, 2020, 2021]

# Create dictionaries to store the combined data for each year
k_dst_combined_dict = {}
complete_dict = {}

for year in years:
    long_kicker = globals()[f'full_kicker_{year}']
    long_defense = globals()[f'full_defense_{year}']
    weekly_performance = globals()[f'weekly_performance_{year}']

    # Combine kicker and defense DataFrames
    k_dst_year = pd.concat([long_kicker, long_defense], axis=0, ignore_index=True, sort=False)
    k_dst_year = k_dst_year.rename(columns={'Team': 'recent_team', 'Pos': 'position', 'Points': 'fantasy_points_ppr', 'Week': 'week', 'Player': 'player_display_name'})

    # Convert data types and handle missing values
    k_dst_year['week'] = k_dst_year['week'].astype('int32')
    k_dst_year['fantasy_points_ppr'] = pd.to_numeric(k_dst_year['fantasy_points_ppr'], errors='coerce').fillna(0).astype('float32')

    # Store combined DataFrame in a dictionary
    k_dst_combined_dict[f'long_combined_{year}'] = k_dst_year

    # Concatenate with the complete DataFrame for the specific year
    complete_year = pd.concat([weekly_performance, k_dst_year], axis=0, ignore_index=True, sort=False)

    # Replace team names in the 'recent_team' column
    complete_year['recent_team'] = complete_year['recent_team'].replace({'JAC': 'JAX', 'LA': 'LAR', 'OAK': 'LV'})

    # Check if "position group" column is empty
    mask = complete_year['position_group'].isnull()

    # Copy value from "position" column to "position group" column where it is empty
    complete_year.loc[mask, 'position_group'] = complete_year.loc[mask, 'position']

    # Only keep the columns that are needed
    complete_year = complete_year[['player_id', 'player_display_name', 'position_group', 'recent_team', 'season', 'week', 'fantasy_points_ppr']]
    
    # Store complete DataFrame in a dictionary
    complete_dict[f'player_fantasy_data_{year}'] = complete_year


In [6]:
# Drop irrelevant positions
relevant_positions = ['QB', 'RB', 'WR', 'TE', 'K', 'DST']

filtered_dict = {}

for year in years:
    complete_data = complete_dict[f'player_fantasy_data_{year}']
    filtered_data = complete_data[complete_data['position_group'].isin(relevant_positions)]
    filtered_dict[f'player_fantasy_data_{year}'] = filtered_data

In [7]:
# Create a dictionary to store average and variance data for each year
fantasy_points_dict = {}

# Calculate average fantasy points and variance for each year
for year in years:
    filtered_data = filtered_dict[f'player_fantasy_data_{year}']
    
    # Calculate average fantasy points and variance for each player in the current year
    player_stats = filtered_data.groupby('player_id')['fantasy_points_ppr'].agg(['mean', 'var']).reset_index()
    
    # Rename columns for clarity
    player_stats.columns = ['player_id', f'average_points_{year}', f'variance_points_{year}']
    
    # Append the calculated stats for the current year to the dictionary
    fantasy_points_dict[year] = player_stats[['player_id', f'average_points_{year}', f'variance_points_{year}']]
    
# Merge the calculated stats with the existing complete_year DataFrame
fantasy_points = complete_year.copy()

for year in years:
    # Merge with the stats for the current year
    fantasy_points = pd.merge(fantasy_points, fantasy_points_dict[year], on='player_id', how='left')

# Filter out positions that are not in relevant_positions
fantasy_points = fantasy_points[fantasy_points['position_group'].isin(relevant_positions)]

# Drop columns for 2021
fantasy_points = fantasy_points.drop(columns=['average_points_2021', 'variance_points_2021'])

In [8]:
# Create a dictionary to store average and variance data for each year
fantasy_points_dict = {}

# Calculate average fantasy points and variance for each year
for year in years:
    filtered_data = filtered_dict[f'player_fantasy_data_{year}']
    
    # Calculate average fantasy points and variance for each player in the current year
    player_stats = filtered_data.groupby('player_id')['fantasy_points_ppr'].agg(['mean', 'var']).reset_index()
    
    # Rename columns for clarity
    player_stats.columns = ['player_id', f'average_points_{year}', f'variance_points_{year}']
    
    # Append the calculated stats for the current year to the dictionary
    fantasy_points_dict[year] = player_stats[['player_id', f'average_points_{year}', f'variance_points_{year}']]
    
# Merge the calculated stats with the existing complete_year DataFrame
fantasy_points = complete_year.copy()

for year in years:
    # Merge with the stats for the current year
    fantasy_points = pd.merge(fantasy_points, fantasy_points_dict[year], on='player_id', how='left')

# Filter out positions that are not in relevant_positions
fantasy_points = fantasy_points[fantasy_points['position_group'].isin(relevant_positions)]

# Drop columns for 2021
fantasy_points = fantasy_points.drop(columns=['average_points_2021', 'variance_points_2021'])

In [9]:
# Schedule data
# Define the years for which you want to import and filter data
years = [2019, 2020, 2021]

# Call the functions to get the filtered data
schedule_2019 = import_schedule_data([2019])
schedule_2020 = import_schedule_data([2020])
schedule_2021 = import_schedule_data([2021])

# Calculating defensive fantasy points allowed
# Create a dictionary to store average fantasy points allowed data for each year
avg_fantasy_points_allowed_dict = {}

# Calculate average fantasy points allowed for each team and position group for each year
for year in years:
    # Retrieve the schedule dataframe for the current year
    schedule = globals()[f'schedule_{year}']

    # Replace team abbreviations in the 'home_team' and 'away_team' columns
    schedule['home_team'] = schedule['home_team'].replace({'JAC': 'JAX', 'LA': 'LAR', 'OAK': 'LV'})
    schedule['away_team'] = schedule['away_team'].replace({'JAC': 'JAX', 'LA': 'LAR', 'OAK': 'LV'})

    # Retrieve the player_fantasy_data dataframe for the current year
    player_fantasy_data = fantasy_points

    # Merge with the schedule dataframe for home teams
    home_games = player_fantasy_data.merge(schedule, left_on=['recent_team', 'week'], right_on=['home_team', 'week'], how='left')

    # Merge with the schedule dataframe for away teams
    away_games = player_fantasy_data.merge(schedule, left_on=['recent_team', 'week'], right_on=['away_team', 'week'], how='left')

    # Create a copy of the original dataframe
    fantasy_data_copy = player_fantasy_data.copy()

    # Reset index for all dataframes
    home_games = home_games.reset_index(drop=True)
    away_games = away_games.reset_index(drop=True)
    fantasy_data_copy = fantasy_data_copy.reset_index(drop=True)

    # Update the opponent column based on home and away teams
    fantasy_data_copy.loc[:, 'opponent'] = home_games['away_team']
    nan_rows = fantasy_data_copy['opponent'].isna()
    fantasy_data_copy.loc[nan_rows, 'opponent'] = away_games.loc[nan_rows, 'home_team']

    # Calculate average fantasy points allowed for each team and position group
    avg_fantasy_points_allowed = fantasy_data_copy.groupby(['opponent', 'position_group'])['fantasy_points_ppr'].mean().reset_index()

    # Rename columns for clarity
    avg_fantasy_points_allowed.columns = ['team', 'position_group', f'avg_fantasy_points_allowed_{year}']

    # Store the calculated stats for the current year in the dictionary
    avg_fantasy_points_allowed_dict[year] = avg_fantasy_points_allowed

    # Filter out rows where 'opponent' is NaN in the fantasy_data_copy dataframe
    fantasy_data_player = fantasy_data_copy.dropna(subset=['opponent'])


In [10]:
# Extract home game information for the schedule
home = schedule[['game_id', 'home_team', 'week']].copy()
home['home_game'] = True

# Extract away game information for the 2020 schedule
away = schedule[['game_id', 'away_team', 'week']].copy()
away['home_game'] = False

# Rename columns for clarity in the home and away dataframes
home.rename(columns={'home_team': 'recent_team'}, inplace=True)
away.rename(columns={'away_team': 'recent_team'}, inplace=True)

# Concatenate home and away dataframes to create a dataframe with all games
all_games = pd.concat([home, away], axis=0, ignore_index=True, sort=False)

# Merge the fantasy_data_player dataframe with the all_games dataframe based on 'recent_team' and 'week'
fantasy_data_player = fantasy_data_player.merge(all_games, on=['recent_team', 'week'], how='left')

# Rename the 'team' column in avg_fantasy_points_allowed_dict[2020] to 'opponent'
avg_fantasy_points_allowed_dict[2020].rename(columns={'team': 'opponent'}, inplace=True)

# Merge the fantasy_data_player dataframe with the avg_fantasy_points_allowed_dict[2020] dataframe
# based on 'opponent' and 'position_group'
fantasy_data_player = fantasy_data_player.merge(avg_fantasy_points_allowed_dict[2020], on=['opponent', 'position_group'], how='left')

# Rename the 'team' column in avg_fantasy_points_allowed_dict[2019] to 'opponent'
avg_fantasy_points_allowed_dict[2019].rename(columns={'team': 'opponent'}, inplace=True)

# Merge the fantasy_data_player dataframe with the avg_fantasy_points_allowed_dict[2019] dataframe
# based on 'opponent' and 'position_group'
fantasy_data_player = fantasy_data_player.merge(avg_fantasy_points_allowed_dict[2019], on=['opponent', 'position_group'], how='left')

In [11]:
# Define the order of columns you want
column_order = ['player_id', 'player_display_name', 'position_group', 'recent_team', 'season', 'week', 'fantasy_points_ppr', 'opponent', 'game_id', 'home_game', 'avg_fantasy_points_allowed_2020', 'avg_fantasy_points_allowed_2019', 'average_points_2019', 'variance_points_2019', 'average_points_2020', 'variance_points_2020']

# Reorder the columns in the DataFrame and uniform names
fantasy_data_player = fantasy_data_player[column_order]
fantasy_data_player['player_display_name'] = fantasy_data_player['player_display_name'].replace({'D.J. ': 'DJ '}, regex=True)

# Impute missing values in average_points and variance_points with the mean of the respective column
fantasy_data_player['average_points_2019'].fillna(fantasy_data_player['average_points_2019'].mean(), inplace=True)
fantasy_data_player['variance_points_2019'].fillna(fantasy_data_player['variance_points_2019'].mean(), inplace=True)

fantasy_data_player['average_points_2020'].fillna(fantasy_data_player['average_points_2020'].mean(), inplace=True)
fantasy_data_player['variance_points_2020'].fillna(fantasy_data_player['variance_points_2020'].mean(), inplace=True)

In [12]:
# Define features and target for the models
features = ['avg_fantasy_points_allowed_2020', 'avg_fantasy_points_allowed_2019', 'average_points_2019', 'variance_points_2019', 'average_points_2020', 'variance_points_2020', 'home_game']
target = 'fantasy_points_ppr'

# Prepare train and test data
X_train, X_test = fantasy_data_player[features], fantasy_data_player[features]
y_train, y_test = fantasy_data_player[target], fantasy_data_player[target]

In [13]:
# Create a Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
prediction_lr = lr_model.predict(X_test)

# Evaluate the model using Mean Absolute Error (MAE) and Mean Squared Error (MSE)
mae = mean_absolute_error(y_test, prediction_lr)
mse = mean_squared_error(y_test, prediction_lr)
rmse = mean_squared_error(y_test, prediction_lr, squared=False)

print("Linear Regression Model")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}\n")

# Add predictions to a copy of the dataframe
fantasy_data_player_lr = fantasy_data_player.copy()
fantasy_data_player_lr['predicted_fantasy_points'] = prediction_lr

Linear Regression Model
Mean Absolute Error: 5.24301290512085
Mean Squared Error: 45.38597106933594
Root Mean Squared Error: 6.736911296844482



In [14]:
# Create a Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=200, random_state=17)
rf_model.fit(X_train, y_train)

# Make predictions
prediction_rf = rf_model.predict(X_test)

# Evaluate the model using Mean Absolute Error (MAE) and Mean Squared Error (MSE)
mae = mean_absolute_error(y_test, prediction_rf)
mse = mean_squared_error(y_test, prediction_rf)
rmse = mean_squared_error(y_test, prediction_rf, squared=False)

print("Random Forest Regressor Model")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}\n")

# Add predictions to a copy of the dataframe
fantasy_data_player_rf = fantasy_data_player.copy()
fantasy_data_player_rf['predicted_fantasy_points'] = prediction_rf

Random Forest Regressor Model
Mean Absolute Error: 2.2429279798369897
Mean Squared Error: 10.299243174651881
Root Mean Squared Error: 3.2092433959816575



In [15]:
# Create an XGBoost model
xgb_model = XGBRegressor(random_state=17)
xgb_model.fit(X_train, y_train)

# Make predictions
prediction_xgb = xgb_model.predict(X_test)

# Evaluate the model using Mean Absolute Error (MAE) and Mean Squared Error (MSE)
mae = mean_absolute_error(y_test, prediction_xgb)
mse = mean_squared_error(y_test, prediction_xgb)
rmse = mean_squared_error(y_test, prediction_xgb, squared=False)

print("XGBoost Model")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}\n")

# Add predictions to a copy of the dataframe
fantasy_data_player_xgb = fantasy_data_player.copy()
fantasy_data_player_xgb['predicted_fantasy_points'] = prediction_xgb

C:\Users\Lenny\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\Lenny\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
C:\Users\Lenny\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
C:\Users\Lenny\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  ret

XGBoost Model
Mean Absolute Error: 3.1939964294433594
Mean Squared Error: 18.82990837097168
Root Mean Squared Error: 4.339344024658203



C:\Users\Lenny\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\Lenny\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
C:\Users\Lenny\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
C:\Users\Lenny\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  ret

In [16]:
# Store as csv files for later import
fantasy_data_player_rf.to_csv('fantasy_data_player_rf.csv', index=False)
fantasy_data_player_xgb.to_csv('fantasy_data_player_xgb.csv', index=False)

In [17]:
# Scraping ADP data
adp_data = scrape_adp_data(2021)

# Add player to the adp_data dataframe
adp_data = pd.merge(adp_data, fantasy_data_player[['player_display_name', 'player_id']], left_on='Player', right_on='player_display_name', how='left')

# Clean up the adp_data dataframe
adp_data = adp_data.drop(columns=['player_display_name'])

# Drop duplicates based on 'player_id'
adp_data = adp_data.drop_duplicates(subset='player_id', keep='first')

In [18]:
# Average projected fantasy points for each player in 2021
average_projections_2021 = fantasy_data_player_rf.groupby(['player_id', 'player_display_name', 'position_group'])['predicted_fantasy_points'].mean().reset_index()
average_projections_2021 = average_projections_2021.sort_values('predicted_fantasy_points', ascending=False)

# Sum of projected fantasy points for each player in 2021
player_projections_2021 = fantasy_data_player_rf.groupby(['player_id', 'player_display_name', 'position_group'])['predicted_fantasy_points'].sum().reset_index()
player_projections_2021 = player_projections_2021.sort_values('predicted_fantasy_points', ascending=False)

# Merge with the adp_data dataframe
adp_data = pd.merge(adp_data, player_projections_2021[['player_id', 'position_group', 'predicted_fantasy_points']], on=['player_id'], how='left')

In [19]:
# Ranking based on projected fantasy points for each player in 2021
# Sort players by position_group and predicted_fantasy_points
position_ranks = adp_data
position_ranks['AVG'] = position_ranks['AVG'].astype('float32')
position_ranks = adp_data.sort_values(by=['position_group', 'predicted_fantasy_points'], ascending=[True, False])

# Set the cutoff point for the VOR model
cutoff_vor = 120

# Select players with AVG <= adp_cutoff
selected_players = position_ranks[position_ranks['AVG'] <= cutoff_vor]

# Find the last selected player for each position
replacement_players = selected_players.groupby('position_group').last()

# Function to calculate replacement value for each player
def calculate_vor(row):
    if row['position_group'] in replacement_players.index:
        return replacement_players.loc[row['position_group'], 'predicted_fantasy_points']
    else:
        return 0

# Apply the replacement_value function to calculate 'replacement_value' for each player
position_ranks['replacement_value'] = position_ranks.apply(calculate_vor, axis=1)

# Calculate 'VOR' for each player
position_ranks['vor'] = position_ranks['predicted_fantasy_points'] - position_ranks['replacement_value']

# Sort players by 'VOR'
ranking_players = position_ranks.sort_values('vor', ascending=False)

# Create a new column 'rank' representing the player's rank based on VOR
ranking_players['rank'] = ranking_players['vor'].rank(ascending=False, method='min')

# Convert 'rank' to float
ranking_players['rank'] = ranking_players['rank'].astype(float)

# Merge the VOR rankings with your existing dataframe
merged_adp_ranking = pd.merge(adp_data, ranking_players[['player_id', 'predicted_fantasy_points', 'position_group', 'replacement_value', 'vor', 'rank']], on='player_id')

# Calculate the average rank based on 'rank' and 'AVG'
merged_adp_ranking['average_rank'] = merged_adp_ranking[['rank', 'AVG']].mean(axis=1)

# Rename columns and drop duplicate columns 
merged_adp_ranking = merged_adp_ranking.rename(columns={'predicted_fantasy_points_x': 'predicted_fantasy_points', 'AVG': 'adp_rank', 'position_group_x': 'position_group'})
merged_adp_ranking = merged_adp_ranking.drop(columns=['predicted_fantasy_points_y', 'position_group_y'])

# Sort the merged dataframe by 'rank'
merged_adp_ranking = merged_adp_ranking.sort_values('rank', ascending=True)

# Print the resulting dataframe
merged_adp_ranking.to_csv('merged_adp_ranks.csv', index=False)


In [20]:
# We will create a copy of the dataframe to avoid modifying the original data
adp_and_vor_data = adp_data.copy()

# Rank players based on ADP ranks
adp_and_vor_data['adp_rank'] = adp_and_vor_data['AVG'].rank(ascending=True, method='min')

# Set the cutoff point for the VOR model
cutoff_vor = 120

# Select players with AVG <= cutoff_vor
selected_players_vor = adp_and_vor_data[adp_and_vor_data['AVG'] <= cutoff_vor]

# Find the last selected player for each position
replacement_players_vor = selected_players_vor.groupby('position_group').last()

# Function to calculate replacement value for each player
def calculate_vor(row):
    if row['position_group'] in replacement_players_vor.index:
        return replacement_players_vor.loc[row['position_group'], 'AVG']
    else:
        return 0

# Apply the replacement_value function to calculate 'replacement_value' for each player
adp_and_vor_data['replacement_value'] = adp_and_vor_data.apply(calculate_vor, axis=1)

# Calculate 'vor' for each player
adp_and_vor_data['vor'] = adp_and_vor_data['predicted_fantasy_points'] - adp_and_vor_data['replacement_value']

# Generate final rankings based on the VOR-adjusted ranks
# Sort players by 'vor'
adp_and_vor_data = adp_and_vor_data.sort_values('vor', ascending=False)

# Create a new column 'rank' representing the player's rank based on VOR
adp_and_vor_data['rank'] = adp_and_vor_data['vor'].rank(ascending=False, method='min')

# Convert 'rank' to float
adp_and_vor_data['rank'] = adp_and_vor_data['rank'].astype(float)

# Print the resulting dataframe
adp_and_vor_data.to_csv('adp_and_vor_ranks.csv', index=False)


In [21]:
# Read CSV file into a DataFrame
consensus_csv = r'C:\Users\Lenny\seminar_paper_flath\datadriven-fantasyfootball\fantasy_football_data\consensus_ranking.csv'
consensus_data = pd.read_csv(consensus_csv, encoding='latin1')

# Extract values inside brackets in the 'PLAYER NAME' column and create a new 'Team' column
consensus_data['Team'] = consensus_data['PLAYER NAME'].str.extract(r'\((.*?)\)')

# Remove brackets and their contents from 'PLAYER NAME' column
consensus_data['PLAYER NAME'] = consensus_data['PLAYER NAME'].replace(r'\(.*\)', '', regex=True).str.strip()

# Remove suffixes like 'Jr.', 'II', and 'III' from 'PLAYER NAME' column
consensus_data['PLAYER NAME'] = consensus_data['PLAYER NAME'].replace(r'\s(Jr\.|II|III)$', '', regex=True).str.strip()
consensus_data['PLAYER NAME'] = consensus_data['PLAYER NAME'].replace({'D.K. ': 'DK '}, regex=True).replace({'D.J. ': 'DJ '}, regex=True)

# Fill empty cells with 'None'
consensus_data['BYE WEEK'].fillna('None', inplace=True)

# Rename columns
consensus_data.rename(columns={'RK': 'consensus_rank', 'PLAYER NAME': 'Player', 'BYE WEEK': 'Bye'}, inplace=True)

# Merge consensus_data with fantasy_data_player to get player_id
consensus_data = pd.merge(consensus_data, fantasy_data_player[['player_display_name', 'player_id']], left_on='Player', right_on='player_display_name', how='left')

# Clean up the consensus_data DataFrame
consensus_data = consensus_data.drop(columns=['player_display_name'])
consensus_data = consensus_data.drop_duplicates(subset='player_id', keep='first')

# Merge projections with consensus rankings
consensus_data = pd.merge(consensus_data, player_projections_2021[['player_id', 'position_group', 'predicted_fantasy_points']], on=['player_id'], how='left')


In [22]:
# Merge the consensus_data with adp_data based on player_id
merged_data = pd.merge(consensus_data, adp_data[['player_id']], on='player_id', how='inner')

# Select only the desired columns from the merged dataframe
final_consensus_data = merged_data[['consensus_rank', 'Player', 'POS', 'Bye', 'Team', 'player_id', 'position_group', 'predicted_fantasy_points']]


In [23]:
# We will create a copy of the dataframe to avoid modifying the original data
consensus_and_vor_data = consensus_data.copy()

# Rank players based on consensus ranks
consensus_and_vor_data['consensus_rank'] = consensus_and_vor_data['consensus_rank'].astype(float)

# Set the cutoff point for the VOR model
cutoff_vor = 120

# Select players with consensus_rank <= cutoff_vor
selected_players_vor = consensus_and_vor_data[consensus_and_vor_data['consensus_rank'] <= cutoff_vor]

# Find the last selected player for each position
replacement_players_vor = selected_players_vor.groupby('position_group').last()

# Function to calculate replacement value for each player
def calculate_vor(row):
    if row['position_group'] in replacement_players_vor.index:
        return replacement_players_vor.loc[row['position_group'], 'consensus_rank']
    else:
        return 0

# Apply the replacement_value function to calculate 'replacement_value' for each player
consensus_and_vor_data['replacement_value'] = consensus_and_vor_data.apply(calculate_vor, axis=1)

# Calculate 'vor' for each player
consensus_and_vor_data['vor'] = consensus_and_vor_data['predicted_fantasy_points'] - consensus_and_vor_data['replacement_value']

# Generate final rankings based on the vor-adjusted ranks
# Sort players by 'vor'
consensus_and_vor_data = consensus_and_vor_data.sort_values('vor', ascending=False)

# Create a new column 'rank' representing the player's rank based on vor
consensus_and_vor_data['rank'] = consensus_and_vor_data['vor'].rank(ascending=False, method='min')

# Convert 'rank' to float
consensus_and_vor_data['rank'] = consensus_and_vor_data['rank'].astype(float)

# Print the resulting dataframe
consensus_and_vor_data.to_csv('consensus_ranks.csv', index=False)

In [24]:
# Define the dataset
opportunity_cost = adp_data.copy()

# Sort players based on projected points for the entire season
opportunity_players = opportunity_cost.sort_values(by=['position_group', 'predicted_fantasy_points'], ascending=[True, False])

# Calculate the opportunity cost for each player, representing the distance to the next best player at the same position
opportunity_players['opportunity_cost'] = opportunity_players.groupby('position_group')['predicted_fantasy_points'].diff(-1).abs()

# Fill NaN values with 0 (for the last player in each position)
opportunity_players['opportunity_cost'].fillna(0, inplace=True)

# Define weights for predicted_fantasy_points and opportunity_cost
weights = {'predicted_fantasy_points': 0.5, 'opportunity_cost': 0.5}

# Calculate the final score using the defined weights
opportunity_players['final_score'] =  opportunity_players['predicted_fantasy_points'] * weights['predicted_fantasy_points'] + \
                                      opportunity_players['opportunity_cost'] * weights['opportunity_cost']

# Sort players by the final score
opportunity_players = opportunity_players.sort_values('final_score', ascending=False)

# Assign ranks based on the final score
opportunity_players['rank'] = opportunity_players['final_score'].rank(ascending=False, method='min')

# Convert 'rank' to float
opportunity_players['rank'] = opportunity_players['rank'].astype(float)

# Merge the final rankings with your existing dataframe
opportunity_cost_data = pd.merge(adp_data, opportunity_players[['player_id', 'predicted_fantasy_points', 'position_group', 'opportunity_cost', 'final_score', 'rank']], on='player_id')

# Rename the columns
opportunity_cost_data = opportunity_cost_data.rename(columns={
    'predicted_fantasy_points_x': 'predicted_fantasy_points',
    'position_group_x': 'position_group'
})

# Drop the unnecessary columns
opportunity_cost_data = opportunity_cost_data.drop(columns=[
    'predicted_fantasy_points_y',
    'position_group_y'
])

# Print the resulting dataframe
opportunity_cost_data.to_csv('opportunity_cost_ranks.csv', index=False)


In [25]:
# Define the dataset (either adp_data or consensus_data)
opportunity_cost_avg = average_projections_2021  # Replace with consensus_data if needed

# Sort players based on projected points for the entire season
opportunity_avg = opportunity_cost_avg.sort_values(by=['position_group', 'predicted_fantasy_points'], ascending=[True, False])

# Calculate the opportunity cost for each player, representing the distance to the next best player at the same position
opportunity_avg['opportunity_cost'] = opportunity_avg.groupby('position_group')['predicted_fantasy_points'].diff(-1).abs()

# Fill NaN values with 0 (for the last player in each position)
opportunity_avg['opportunity_cost'].fillna(0, inplace=True)

# Define weights for predicted_fantasy_points and opportunity_cost
weights = {'predicted_fantasy_points': 0.5, 'opportunity_cost': 0.5}

# Calculate the final score using the defined weights
opportunity_avg['final_score'] = opportunity_avg['predicted_fantasy_points'] * weights['predicted_fantasy_points'] + \
                                 opportunity_avg['opportunity_cost'] * weights['opportunity_cost']

# Sort players by the final score
players = opportunity_avg.sort_values('final_score', ascending=False)

# Assign ranks based on the final score
players['rank'] = players['final_score'].rank(ascending=False, method='min')

# Convert 'rank' to float
players['rank'] = players['rank'].astype(float)

# Merge the final rankings with your existing dataframe
opportunity_cost_avg_data = pd.merge(adp_data, players[['player_id', 'predicted_fantasy_points', 'position_group', 'opportunity_cost', 'final_score', 'rank']], on='player_id')

# Rename the columns
opportunity_cost_avg_data = opportunity_cost_avg_data.rename(columns={
    'predicted_fantasy_points_x': 'predicted_fantasy_points',
    'position_group_x': 'position_group'
})

# Drop the unnecessary columns
opportunity_cost_avg_data = opportunity_cost_avg_data.drop(columns=[
    'predicted_fantasy_points_y',
    'position_group_y'
])

# Print the resulting dataframe
opportunity_cost_avg_data.to_csv('opportunity_cost_avg_ranks.csv', index=False)